In [144]:
import pandas as pd
from datetime import datetime
import numpy as np
from itertools import dropwhile

In [145]:

df = pd.DataFrame([[10  ,      "Mumbai"   ,  '2015-12-11'  ]   ,               
   [1 ,       "Bangalore" , '2016-01-15'],
   [12    ,    "Pune"    ,   '2016-01-26'],
   [4    ,    "Mumbai"  ,    np.nan],
   [15   ,     "Delhi" ,     '2016-04-29'],
   [6   ,     "Mumbai"   ,    np.nan],
   [14    ,    "Bangalore" ,   np.nan],
  [ 5   ,     "Pune"    ,   '2016-05-25'],
   [8   ,     "Mumbai"  ,   '2016-12-13']], columns = ['value', 'city', 'date'])

In [146]:
df

,value,city,date
0,10,Mumbai,2015-12-11
1,1,Bangalore,2016-01-15
2,12,Pune,2016-01-26
3,4,Mumbai,NaN
4,15,Delhi,2016-04-29
5,6,Mumbai,NaN
6,14,Bangalore,NaN
7,5,Pune,2016-05-25
8,8,Mumbai,2016-12-13


In [147]:
df['date'] = pd.to_datetime(df['date'])

In [148]:
df

,value,city,date
0,10,Mumbai,2015-12-11
1,1,Bangalore,2016-01-15
2,12,Pune,2016-01-26
3,4,Mumbai,NaT
4,15,Delhi,2016-04-29
5,6,Mumbai,NaT
6,14,Bangalore,NaT
7,5,Pune,2016-05-25
8,8,Mumbai,2016-12-13


In [149]:
x = df['date'][0]
y = df['date'][3]
x

Timestamp('2015-12-11 00:00:00')

In [150]:
l = df['date'].to_list()
l

[Timestamp('2015-12-11 00:00:00'),
 Timestamp('2016-01-15 00:00:00'),
 Timestamp('2016-01-26 00:00:00'),
 NaT,
 Timestamp('2016-04-29 00:00:00'),
 NaT,
 NaT,
 Timestamp('2016-05-25 00:00:00'),
 Timestamp('2016-12-13 00:00:00')]

In [151]:
i = 0
pairs = []
while i < len(l):
    if pd.isnull(l[i]):
        start = l[i-1]
        j = 0
        while pd.isnull(l[i+j]):
            j += 1
        i += j
        end = l[i]
        pairs.append([start, end, j])
    else:
        i += 1

In [152]:
np.linspace(1, 5, 4)

array([1.        , 2.33333333, 3.66666667, 5.        ])

In [153]:
l1 = [np.linspace(pair[0].timestamp(), pair[1].timestamp(), pair[2]+2) for pair in pairs]
l1

[array([1.4537664e+09, 1.4578272e+09, 1.4618880e+09]),
 array([1.4618880e+09, 1.4626368e+09, 1.4633856e+09, 1.4641344e+09])]

In [154]:
x= []
for i in l1:
    y = []
    for j in i:
        y.append(datetime.fromtimestamp(int(j)).date())
    x.append(y[1:-1])


In [155]:
x

[[datetime.date(2016, 3, 13)],
 [datetime.date(2016, 5, 7), datetime.date(2016, 5, 16)]]

In [156]:
final =[]
for i in x:
    for j in i:
        final.append(j)

In [157]:
final

[datetime.date(2016, 3, 13),
 datetime.date(2016, 5, 7),
 datetime.date(2016, 5, 16)]

In [158]:
final = [pd.to_datetime(i) for i in final]
final

[Timestamp('2016-03-13 00:00:00'),
 Timestamp('2016-05-07 00:00:00'),
 Timestamp('2016-05-16 00:00:00')]

In [159]:
new = []
idx = 0
for i in l:
    if pd.isnull(i):
        new.append(final[idx])
        idx += 1
    else:
        new.append(i)

new

[Timestamp('2015-12-11 00:00:00'),
 Timestamp('2016-01-15 00:00:00'),
 Timestamp('2016-01-26 00:00:00'),
 Timestamp('2016-03-13 00:00:00'),
 Timestamp('2016-04-29 00:00:00'),
 Timestamp('2016-05-07 00:00:00'),
 Timestamp('2016-05-16 00:00:00'),
 Timestamp('2016-05-25 00:00:00'),
 Timestamp('2016-12-13 00:00:00')]

In [160]:
df['date'] = pd.Series(new)

In [161]:
df

,value,city,date
0,10,Mumbai,2015-12-11
1,1,Bangalore,2016-01-15
2,12,Pune,2016-01-26
3,4,Mumbai,2016-03-13
4,15,Delhi,2016-04-29
5,6,Mumbai,2016-05-07
6,14,Bangalore,2016-05-16
7,5,Pune,2016-05-25
8,8,Mumbai,2016-12-13


In [162]:
from tsa.interpolation import interpolate_dates as interpolate

In [163]:
new_df = interpolate(df, 'date', ['value'], interval='D')

In [164]:
new_df

,value,date
0,10.000000,2015-12-11
1,9.742857,2015-12-12
2,9.485714,2015-12-13
3,9.228571,2015-12-14
4,8.971429,2015-12-15
...,...,...
364,7.940594,2016-12-09
365,7.955446,2016-12-10
366,7.970297,2016-12-11
367,7.985149,2016-12-12


In [170]:
from tsa.properties import Seasonality

s = Seasonality(new_df, 'value','date' )
s.seasonal()

date
2015-12-11    0.992910
2015-12-12    0.996281
2015-12-13    1.001242
2015-12-14    1.005790
2015-12-15    1.005674
                ...   
2016-12-09    0.992910
2016-12-10    0.996281
2016-12-11    1.001242
2016-12-12    1.005790
2016-12-13    1.005674
Name: seasonal, Length: 369, dtype: float64